In [30]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

In [50]:
season_list = ['1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']
league_list = ['E0'] 

df_list = []

for league in league_list:
    for season in season_list:
        data_url = f'https://www.football-data.co.uk/mmz4281/{season}/{league}.csv'
        try:
            df = pd.read_csv(data_url)
        except:
            try:
                df = pd.read_csv(data_url, encoding='latin')
            except:
                print(f"Failed to load data for {league} {season}")
                continue
        df['Season'] = season
        df['League'] = league
        df_list.append(df)

main_df = pd.concat(df_list, ignore_index=True)

main_df['Date'] = pd.to_datetime(main_df['Date'], dayfirst=True, errors='coerce')
main_df = main_df[main_df['Date'].notna()]
main_df = main_df.sort_values(by='Date')

df = main_df
df = df[::-1]
df.reset_index(inplace=True)

/var/folders/rt/kk5y63zd2dl20zp4x97pdp2m0000gn/T/ipykernel_8739/1514236676.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  main_df['Date'] = pd.to_datetime(main_df['Date'], dayfirst=True, errors='coerce')


In [33]:
import pandas as pd

season_list = ['1617', '1718', '1819', '1920', '2021', '2122', '2223', '2324', '2425']

def get_teams_by_league(league):
    df_list = []

    for season in season_list:
        url = f"https://www.football-data.co.uk/mmz4281/{season}/{league}.csv"
        try:
            df = pd.read_csv(url)
        except:
            try:
                df = pd.read_csv(url, encoding='latin')
            except:
                print(f"❌ Misslyckades att ladda {league} {season}")
                continue

        df_list.append(df)

    if not df_list:
        return []

    full_df = pd.concat(df_list, ignore_index=True)

    # Plocka ut alla unika lag från både hemma och bortalag
    teams = pd.unique(pd.concat([full_df['HomeTeam'], full_df['AwayTeam']]).dropna())
    return sorted(teams.tolist())

In [34]:
df_swe = pd.read_csv('https://www.football-data.co.uk/new/SWE.csv')

df_swe['Date'] = pd.to_datetime(df_swe['Date'], format='%d/%m/%Y')
df_swe= df_swe.sort_values(by='Date')
df_swe = df_swe[::-1]

df_swe.reset_index(inplace=True)

In [35]:
def get_teams_result(df):
    resultat = {}

    teams = set(df["HomeTeam"]).union(set(df["AwayTeam"]))

    for team in teams:
        resultat[team] = {}
        
    for _, rad in df.iterrows():
        hemma, borta = rad["HomeTeam"], rad["AwayTeam"]
        mål_a, mål_b = rad["FTHG"], rad["FTAG"]

        if hemma not in resultat:
            resultat[hemma] = {}
        if borta not in resultat:
            resultat[borta] = {}

        if borta not in resultat[hemma]:
            resultat[hemma][borta] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}
        if hemma not in resultat[borta]:
            resultat[borta][hemma] = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

        if mål_a > mål_b: 
            resultat[hemma][borta]["Vinster"] += 1
            resultat[borta][hemma]["Förluster"] += 1
        elif mål_a < mål_b: 
            resultat[borta][hemma]["Vinster"] += 1
            resultat[hemma][borta]["Förluster"] += 1
        else:
            resultat[hemma][borta]["Oavgjorda"] += 1
            resultat[borta][hemma]["Oavgjorda"] += 1
            
    return resultat

#get_teams_result(df)['Man City']

In [56]:
def get_home_vs_away_summary(df, home_team, away_team):

    matcher = df[(df["HomeTeam"] == home_team) & (df["AwayTeam"] == away_team)]

    resultat = {"Vinster": 0, "Förluster": 0, "Oavgjorda": 0}

    for _, rad in matcher.iterrows():
        mål_hemma = rad["FTHG"]
        mål_borta = rad["FTAG"]

        if mål_hemma > mål_borta:
            resultat["Vinster"] += 1
        elif mål_hemma < mål_borta:
            resultat["Förluster"] += 1
        else:
            resultat["Oavgjorda"] += 1

    return resultat

get_home_vs_away_summary(df,'Arsenal', 'Liverpool')

{'Vinster': 3, 'Förluster': 3, 'Oavgjorda': 3}

In [37]:
def get_team_league_test():
    teams = []

    # Hantera engelska ligor (Div)
    all_eng_teams = df['HomeTeam'].astype(str).tolist() + df['AwayTeam'].astype(str).tolist()   
    unique_eng_teams = np.unique(all_eng_teams)
    teams.extend([str(team) for team in unique_eng_teams])

    # Hämta ligor från båda dataframes
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    swedish_leagues = sorted(df_swe['League'].astype(str).unique().tolist())

    all_leagues = english_leagues + swedish_leagues
    return teams, all_leagues

In [38]:
def get_team_league():
    teams_by_league = []
    all_leagues = []

    # Hantera engelska ligor
    english_leagues = sorted(df['Div'].astype(str).unique().tolist())
    for league in english_leagues:
        league_df = df[df['Div'] == league]
        teams = league_df['HomeTeam'].astype(str).tolist() + league_df['AwayTeam'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    # Hantera svenska ligor, ta bort whitespaces och duplicates
    swedish_leagues_raw = df_swe['League'].astype(str).str.strip()
    swedish_leagues = sorted(swedish_leagues_raw.unique().tolist())

    for league in swedish_leagues:
        league_df = df_swe[df_swe['League'].str.strip() == league]
        teams = league_df['Home'].astype(str).tolist() + league_df['Away'].astype(str).tolist()
        unique_teams = sorted(np.unique(teams))
        teams_by_league.append(unique_teams)
        all_leagues.append(league)

    return teams_by_league, all_leagues

In [39]:
col_list = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'B365H', 'B365D', 'B365A']

In [40]:
def get_team_form(df, team, num_matches=10):

    team_matches = df[(df['HomeTeam'] == team) | (df['AwayTeam'] == team)].copy()

    team_matches = team_matches.sort_values(by='Date', ascending=False).head(num_matches)

    form = []

    for _, row in team_matches.iterrows():
        is_home = row['HomeTeam'] == team
        goals_for = row['FTHG'] if is_home else row['FTAG']
        goals_against = row['FTAG'] if is_home else row['FTHG']

        if goals_for > goals_against:
            form.append('W')
        elif goals_for == goals_against:
            form.append('D')
        else:
            form.append('L')

    return form[::-1]  

#get_team_form(df, 'Tottenham')

In [41]:
def get_fixture_by_league(league):
    df_next_games = pd.read_csv('https://www.football-data.co.uk/fixtures.csv')
    
    df_next_games['Date'] = pd.to_datetime(df_next_games['Date'], format='%d/%m/%Y', errors='coerce')
    today = pd.Timestamp.today().normalize()
    df_next_games = df_next_games[df_next_games['Date'] >= today]

    if league in df_next_games['Div'].unique():
        league_fixtures = df_next_games[df_next_games['Div'] == league]
        fixtures_list = league_fixtures[['HomeTeam', 'AwayTeam', 'Date', 'Time']].dropna().to_dict(orient='records')
        return fixtures_list
    return []

In [42]:
get_fixture_by_league('E0')

[{'HomeTeam': 'Nottingham',
  'AwayTeam': 'Brentford',
  'Date': Timestamp('2025-05-01 00:00:00'),
  'Time': '19:30'}]

In [43]:
def create_league_table_for_current_year(league, df):
    current_year = datetime.now().year
    
    if datetime.now().month < 8:
        start_date = f"{current_year - 1}-08-01" 
        end_date = f"{current_year}-07-01"   
    else:
        start_date = f"{current_year}-08-01"      
        end_date = f"{current_year + 1}-07-01"   

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    matches_df = df[(df['Div'] == league) & 
                    (pd.to_datetime(df['Date']) >= start_date) & 
                    (pd.to_datetime(df['Date']) < end_date)]
    
    return matches_df

In [44]:
create_league_table_for_current_year('E0', df).tail()

,index,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Season,League,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFH,BFD,BFA,1XBH,1XBD,1XBA,BFEH,BFED,BFEA,BFE>2.5,BFE<2.5,BFEAHH,BFEAHA,BFCH,BFCD,BFCA,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
334,3043,E0,2024-08-17,Everton,Brighton,0,3,A,0,1,A,S Hooper,9,10,1,5,8,8,1,5,1,1,1,0,2.63,3.3,2.63,2.65,3.4,2.60,NaN,NaN,NaN,NaN,NaN,NaN,2.73,3.36,2.71,2.60,3.5,2.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.15,3.41,2.40,2425,E0,15:00,2.76,3.66,2.78,2.67,3.41,2.68,1.80,2.00,1.83,2.05,1.85,2.08,1.81,2.04,0.0,1.96,1.97,1.96,1.94,1.97,1.97,1.94,1.94,3.10,3.40,2.30,3.00,3.40,2.37,NaN,NaN,NaN,3.10,3.3,2.40,NaN,NaN,NaN,3.16,3.50,2.45,3.06,3.40,2.38,1.93,1.97,1.93,1.97,1.95,2.00,1.89,1.96,0.25,1.87,2.06,1.86,2.07,1.92,2.10,1.83,2.04,2.70,3.40,2.63,2.68,3.66,2.63,2.78,3.5,2.78,1.88,2.08,1.99,1.99,3.00,3.3,2.30,3.16,3.47,2.34,3.15,3.55,2.46,1.94,2.04,1.88,2.11
335,3042,E0,2024-08-17,Arsenal,Wolves,2,0,H,1,0,H,J Gillett,18,9,6,3,17,14,8,2,2,2,0,0,1.18,7.5,13.00,1.20,7.5,13.50,NaN,NaN,NaN,NaN,NaN,NaN,1.16,8.56,16.22,1.18,7.0,17.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.15,9.05,18.76,2425,E0,15:00,1.20,9.10,18.00,1.18,7.86,15.87,1.44,2.75,1.46,2.79,1.50,2.82,1.46,2.70,-2.0,1.93,2.00,1.88,2.00,1.97,2.00,1.90,1.96,1.14,8.50,15.00,1.16,8.50,18.00,NaN,NaN,NaN,1.15,8.0,19.00,NaN,NaN,NaN,1.17,9.40,21.00,1.15,8.62,18.11,1.40,3.00,1.41,2.98,1.45,3.00,1.42,2.93,-2.25,2.02,1.91,2.00,1.90,2.05,1.93,1.99,1.87,1.15,8.00,16.00,1.20,7.65,16.00,1.19,9.0,18.00,1.53,2.84,1.94,2.00,1.13,8.5,17.00,1.16,9.39,16.60,1.17,9.40,21.00,1.44,3.20,2.02,1.96
336,3041,E0,2024-08-17,Ipswich,Liverpool,0,2,A,0,0,D,T Robinson,7,18,2,5,9,18,2,10,3,1,0,0,8.50,5.5,1.33,7.50,5.5,1.36,NaN,NaN,NaN,NaN,NaN,NaN,8.18,5.84,1.34,8.50,5.5,1.33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.14,6.09,1.34,2425,E0,12:30,9.00,6.10,1.37,8.28,5.76,1.34,1.40,3.00,1.41,3.00,1.43,3.07,1.41,2.94,1.5,2.02,1.91,1.99,1.92,2.02,1.95,1.97,1.90,8.00,5.75,1.33,8.00,5.75,1.34,NaN,NaN,NaN,8.00,5.5,1.35,NaN,NaN,NaN,8.57,6.25,1.39,7.87,5.81,1.35,1.36,3.20,1.37,3.30,1.40,3.38,1.37,3.18,1.50,2.05,1.88,2.04,1.90,2.20,2.00,1.99,1.88,8.50,5.50,1.33,8.60,5.85,1.35,9.40,6.2,1.36,1.45,3.15,2.04,1.93,7.50,5.5,1.33,8.57,5.85,1.34,8.60,6.20,1.37,1.40,3.40,2.04,1.93
337,3045,E0,2024-08-17,Nott'm Forest,Bournemouth,1,1,D,1,0,H,M Oliver,14,13,8,4,17,8,2,6,1,3,0,0,2.45,3.5,2.80,2.45,3.5,2.80,NaN,NaN,NaN,NaN,NaN,NaN,2.47,3.42,2.97,2.50,3.5,2.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.24,3.50,3.37,2425,E0,15:00,2.51,3.70,3.00,2.45,3.49,2.89,1.73,2.10,1.79,2.11,1.81,2.14,1.77,2.08,0.0,1.82,2.11,1.78,2.14,1.83,2.14,1.80,2.09,2.20,3.40,3.25,2.20,3.40,3.25,NaN,NaN,NaN,2.25,3.4,3.25,NaN,NaN,NaN,2.38,3.55,3.37,2.25,3.44,3.23,1.89,2.01,1.89,2.02,1.93,2.04,1.87,1.96,-0.25,1.94,1.99,1.94,1.98,2.05,2.00,1.93,1.93,2.45,3.50,2.88,2.46,3.70,2.87,2.54,3.6,3.00,1.83,2.16,1.83,2.17,2.20,3.3,3.25,2.26,3.49,3.33,2.30,3.50,3.50,1.91,2.08,1.96,2.02
338,3040,E0,2024-08-16,Man United,Fulham,1,0,H,0,0,D,R Jones,14,10,5,2,12,10,7,8,2,3,0,0,1.60,4.2,5.25,1.60,4.4,5.25,NaN,NaN,NaN,NaN,NaN,NaN,1.63,4.38,5.30,1.65,4.2,5.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [ ]:
import pandas as pd

def get_avg_goals_for_home_and_away(league, home_team, away_team):
    matches_df = create_league_table_for_current_year(league, df)
    print(matches_df.tail())
    
    home_matches = matches_df[matches_df['HomeTeam'] == home_team]
    away_matches = matches_df[matches_df['AwayTeam'] == away_team]

    if len(home_matches) > 0:
        home_goals_avg = home_matches['FTHG'].mean() 
    else:
        home_goals_avg = 0


    if len(away_matches) > 0:
        away_goals_avg = away_matches['FTAG'].mean() 
    else:
        away_goals_avg = 0
    
    return {
        'home_team_avg_goals': home_goals_avg,
        'away_team_avg_goals': away_goals_avg
    }


In [46]:

def get_last_five_results(df: pd.DataFrame) -> list[dict]:
    # Säkerställ att kolumner vi behöver finns
    required_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("❌ DataFrame saknar nödvändiga kolumner: " + ", ".join(required_columns))
    
    # Konvertera datum
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date'])

    # Sortera efter datum baklänges
    df_sorted = df.sort_values('Date', ascending=False)

    # Ta de 5 senaste matcherna
    last_five = df_sorted.head(5)

    # Skapa en lista med matchresultaten
    results = []
    for _, row in last_five.iterrows():
        results.append({
            "date": row['Date'].strftime('%Y-%m-%d'),
            "home_team": row['HomeTeam'],
            "away_team": row['AwayTeam'],
            "home_goals": int(row['FTHG']),
            "away_goals": int(row['FTAG']),
            "result": row['FTR']  # Full Time Result: H = Home Win, A = Away Win, D = Draw
        })

    return results

In [61]:
get_avg_goals_for_home_and_away('E0', 'Arsenal', 'Aston Villa')

{'home_team_avg_goals': np.float64(1.9411764705882353),
 'away_team_avg_goals': np.float64(1.3529411764705883)}

In [48]:
def get_home_and_away_form(df, home_team, away_team, num_matches=10):
    home_matches = df[df["HomeTeam"] == home_team].sort_values(by="Date", ascending=False).head(num_matches)
    away_matches = df[df["AwayTeam"] == away_team].sort_values(by="Date", ascending=False).head(num_matches)

    home_form = []
    for _, row in home_matches.iterrows():
        if row["FTHG"] > row["FTAG"]:
            home_form.append("W")
        elif row["FTHG"] == row["FTAG"]:
            home_form.append("D")
        else:
            home_form.append("L")

    away_form = []
    for _, row in away_matches.iterrows():
        if row["FTAG"] > row["FTHG"]:
            away_form.append("W")
        elif row["FTAG"] == row["FTHG"]:
            away_form.append("D")
        else:
            away_form.append("L")

    return {"home_form": home_form[::-1], "away_form": away_form[::-1]}

def get_team_data(league_df, home_team, away_team):
    response = {}
    avg_goals = get_avg_goals_for_home_and_away(league_df, home_team, away_team)
    response["avg_goals"] = avg_goals

    home_away_form = get_home_and_away_form(league_df, home_team, away_team)
    response["home_away_form"] = home_away_form

    # ... rest of the existing code in get_team_data ...
    return response

In [49]:

def get_last_five_results(df: pd.DataFrame) -> list[dict]:
    # Säkerställ att kolumner vi behöver finns
    required_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
    if not all(col in df.columns for col in required_columns):
        raise ValueError("❌ DataFrame saknar nödvändiga kolumner: " + ", ".join(required_columns))x
    
    # Konvertera datum
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date'])

    # Sortera efter datum baklänges
    df_sorted = df.sort_values('Date', ascending=False)

    # Ta de 5 senaste matcherna
    last_five = df_sorted.head(5)

    # Skapa en lista med matchresultaten
    results = []
    for _, row in last_five.iterrows():
        results.append({
            "date": row['Date'].strftime('%Y-%m-%d'),
            "home_team": row['HomeTeam'],
            "away_team": row['AwayTeam'],
            "home_goals": int(row['FTHG']),
            "away_goals": int(row['FTAG']),
            "result": row['FTR']  # Full Time Result: H = Home Win, A = Away Win, D = Draw
        })

    return results

SyntaxError: invalid syntax (4259248401.py, line 5)

In [ ]:
df.tail()

,index,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,VCH,VCD,VCA,Bb1X2,BbMxH,BbAvH,BbMxD,BbAvD,BbMxA,BbAvA,BbOU,BbMx>2.5,BbAv>2.5,BbMx<2.5,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA,Season,League,Time,MaxH,MaxD,MaxA,AvgH,AvgD,AvgA,B365>2.5,B365<2.5,P>2.5,P<2.5,Max>2.5,Max<2.5,Avg>2.5,Avg<2.5,AHh,B365AHH,B365AHA,PAHH,PAHA,MaxAHH,MaxAHA,AvgAHH,AvgAHA,B365CH,B365CD,B365CA,BWCH,BWCD,BWCA,IWCH,IWCD,IWCA,WHCH,WHCD,WHCA,VCCH,VCCD,VCCA,MaxCH,MaxCD,MaxCA,AvgCH,AvgCD,AvgCA,B365C>2.5,B365C<2.5,PC>2.5,PC<2.5,MaxC>2.5,MaxC<2.5,AvgC>2.5,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,BFH,BFD,BFA,1XBH,1XBD,1XBA,BFEH,BFED,BFEA,BFE>2.5,BFE<2.5,BFEAHH,BFEAHA,BFCH,BFCD,BFCA,1XBCH,1XBCD,1XBCA,BFECH,BFECD,BFECA,BFEC>2.5,BFEC<2.5,BFECAHH,BFECAHA
3374,61,E0,2016-01-10,Hull,Chelsea,0,2,A,0,0,D,A Taylor,8,22,3,9,13,15,5,7,2,2,0,0,7.50,4.50,1.50,7.00,4.50,1.45,6.5,4.20,1.50,7.00,4.40,1.50,7.68,4.63,1.48,7.0,4.33,1.50,7.50,4.60,1.50,56.0,7.68,6.93,4.75,4.45,1.54,1.49,49.0,1.77,1.72,2.20,2.11,35.0,1.00,2.19,2.09,1.80,1.77,7.50,4.90,1.46,1617,E0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3375,62,E0,2016-01-10,Sunderland,West Brom,1,1,D,0,1,A,S Attwell,7,17,2,7,7,13,6,5,1,3,0,0,2.50,3.25,3.20,2.45,3.10,3.10,2.5,3.30,2.85,2.40,3.20,3.25,2.47,3.24,3.26,2.5,3.10,3.20,2.45,3.25,3.25,55.0,2.55,2.44,3.30,3.17,3.30,3.16,48.0,2.40,2.25,1.70,1.64,32.0,-0.25,2.17,2.05,1.86,1.81,2.54,3.17,3.21,1617,E0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3376,63,E0,2016-01-10,Swansea,Liverpool,1,2,A,1,0,H,M Oliver,8,18,3,6,11,9,3,10,2,2,0,0,8.00,4.75,1.45,7.00,4.50,1.45,7.0,4.50,1.45,7.50,4.75,1.44,8.44,4.92,1.43,8.0,4.50,1.44,8.00,4.80,1.44,56.0,8.44,7.38,5.14,4.72,1.49,1.44,45.0,1.61,1.55,2.54,2.42,35.0,1.00,2.34,2.24,1.72,1.67,7.30,5.04,1.46,1617,E0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3377,65,E0,2016-01-10,West Ham,Middlesbrough,1,1,D,0,0,D,N Swarbrick,19,9,2,3,13,12,4,5,2,3,0,0,2.25,3.40,3.50,2.10,3.40,3.60,2.1,3.40,3.50,2.20,3.40,3.50,2.23,3.35,3.66,2.3,3.30,3.30,2.20,3.40,3.60,55.0,2.30,2.21,3.50,3.31,3.66,3.50,49.0,2.15,2.09,1.85,1.74,32.0,-0.25,1.95,1.88,2.04,1.97,2.00,3.52,4.26,1617,E0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3378,64,E0,2016-01-10,Watford,Bournemouth,2,2,D,0,1,A,M Dean,17,11,7,2,17,12,4,5,3,4,0,0,2.38,3.30,3.30,2.30,3.25,3.25,2.2,3.15,3.50,2.38,3.25,3.25,2.42,3.32,3.27,2.4,3.20,3.20,2.38,3.40,3.25,55.0,2.42,2.35,3.45,3.28,3.50,3.21,49.0,2.09,2.00,1.89,1.81,32.0,-0.25,2.06,2.00,1.91,1.86,2.51,3.33,3.11,1617,E0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
get_last_five_results(df)

NameError: name 'get_last_five_results' is not defined

In [58]:
def get_avg_goals_in_head_to_head(df, league, home_team, away_team):
    # Filtrera endast matcher i rätt liga där home_team mött away_team
    head_to_head = df[
        (df['Div'] == league) &
        (df['HomeTeam'] == home_team) &
        (df['AwayTeam'] == away_team)
    ]

    if len(head_to_head) > 0:
        avg_home_goals = head_to_head['FTHG'].mean()  # Hemmalagets mål i dessa möten
        avg_away_goals = head_to_head['FTAG'].mean()  # Bortalagets mål i dessa möten
    else:
        avg_home_goals = 0
        avg_away_goals = 0

    return {
        'avg_home_goals_vs_away': avg_home_goals,
        'avg_away_goals_vs_home': avg_away_goals
    }

In [82]:
get_avg_goals_in_head_to_head(df, 'E0', 'Arsenal', 'Tottenham', )

{'avg_home_goals_vs_away': np.float64(2.3333333333333335),
 'avg_away_goals_vs_home': np.float64(1.2222222222222223)}

In [69]:
def get_teames(df):
    teams = pd.unique(pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna())

    return teams

In [76]:
def create_league_table_for_current_year(league, df):
    current_year = datetime.now().year
    
    if datetime.now().month < 8:
        start_date = f"{current_year - 1}-08-01" 
        end_date = f"{current_year}-07-01"   
    else:
        start_date = f"{current_year}-08-01"      
        end_date = f"{current_year + 1}-07-01"   

    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    matches_df = df[(df['Div'] == league) & 
                    (pd.to_datetime(df['Date']) >= start_date) & 
                    (pd.to_datetime(df['Date']) < end_date)]
    
    return matches_df

In [81]:
def get_top_avg_goals_per_team(league: str, df: pd.DataFrame, top_n: int = 10) -> pd.DataFrame:
    """
    Returnerar en topplista över lag sorterade på snittmål per match i aktuell säsong.
    """
    df = create_league_table_for_current_year(league, df)

    df = df.dropna(subset=['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG'])
    df['FTHG'] = pd.to_numeric(df['FTHG'], errors='coerce')
    df['FTAG'] = pd.to_numeric(df['FTAG'], errors='coerce')

    teams = pd.unique(pd.concat([df['HomeTeam'], df['AwayTeam']]).dropna())
    team_stats = []

    for team in teams:
        home_matches = df[df['HomeTeam'] == team]
        away_matches = df[df['AwayTeam'] == team]

        home_goals = home_matches['FTHG'].sum()
        away_goals = away_matches['FTAG'].sum()

        home_played = len(home_matches)
        away_played = len(away_matches)

        total_goals = home_goals + away_goals
        total_matches = home_played + away_played
        avg_goals = total_goals / total_matches if total_matches > 0 else 0

        team_stats.append({
            'Team': team,
            'Goals': total_goals,
            'Matches': total_matches,
            'AvgGoalsPerMatch': round(avg_goals, 2)
        })

    top_df = pd.DataFrame(team_stats).sort_values(by='AvgGoalsPerMatch', ascending=False).head(top_n)
    return top_df.reset_index(drop=True)

In [80]:
get_top_avg_goals_per_team('E0', df)

,Team,Goals,Matches,AvgGoalsPerMatch
0,Liverpool,80,34,2.35
1,Man City,66,34,1.94
2,Newcastle,65,34,1.91
3,Arsenal,63,34,1.85
4,Tottenham,62,34,1.82
5,Chelsea,59,34,1.74
6,Brentford,56,33,1.70
7,Brighton,56,34,1.65
8,Nott'm Forest,53,33,1.61
9,Aston Villa,54,34,1.59
